strangely enough, the most accurate predictor was knn with location by itself (lat and long) with a score of 83%.  adding extra variables only reduced the score.  furthermore, using a randomforest also returned a lower score of 68%. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In C:\Users\fawaz\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\fawaz\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\fawaz\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\fawaz\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\fawaz\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

In [24]:
df = pd.read_csv('US_Accidents_June20.csv', parse_dates = ['Start_Time', 'End_Time'])
df.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Number,Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,I-70 E,R,Dayton,Montgomery,OH,45424,US,US/Eastern,KFFO,2016-02-08 05:58:00,36.9,NaN,91.0,29.68,10.0,Calm,NaN,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,2584.0,Brice Rd,L,Reynoldsburg,Franklin,OH,43068-3402,US,US/Eastern,KCMH,2016-02-08 05:51:00,37.9,NaN,100.0,29.65,10.0,Calm,NaN,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,Accident on OH-32 State Route 32 Westbound at ...,NaN,State Route 32,R,Williamsburg,Clermont,OH,45176,US,US/Eastern,KI69,2016-02-08 06:56:00,36.0,33.3,100.0,29.67,10.0,SW,3.5,NaN,Overcast,False,False,False,False,False,False,False,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,I-75 S,R,Dayton,Montgomery,OH,45417,US,US/Eastern,KDAY,2016-02-08 07:38:00,35.1,31.0,96.0,29.64,9.0,SW,4.6,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,NaN,Miamisburg Centerville Rd,R,Dayton,Montgomery,OH,45459,US,US/Eastern,KMGY,2016-02-08 07:53:00,36.0,33.3,89.0,29.65,6.0,SW,3.5,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,True,False,Day,Day,Day,Day


In [23]:
df['month'] = df['Start_Time'].dt.month
df['year'] = df['Start_Time'].dt.year
df['hour'] = df['Start_Time'].dt.hour
df['day'] = df['Start_Time'].dt.weekday

#print(df['year'][:10])

year_ticks = [2016, 2017, 2018, 2019, 2020]

In [3]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors = 10)
x = df.Start_Lat
y = df.Start_Lng

coords = [(x[i], y[i]) for i in range(0, len(x))] 

In [4]:
sev = df.Severity
model.fit(coords, sev)

KNeighborsClassifier(n_neighbors=10)

In [5]:
test_list = [(32.768646, -96.804612), (32.821100, -96.761431)]
res = model.predict(test_list)
print(res)

[3 2]


In [6]:
from sklearn.model_selection import train_test_split

knn = KNeighborsClassifier(n_neighbors = 10)
[]
X_train, X_test, y_train, y_test = train_test_split(coords, sev, test_size=0.4, random_state=22)

In [7]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=10)

In [8]:
from sklearn import metrics
test = knn.predict(X_test)
#test.reshape(-1, 1)
score = metrics.accuracy_score(y_test, test)

print(score)

0.8281194523877456


In [9]:
k_range = np.arange(2, 10)
scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    scored = metrics.accuracy_score(y_test, y_pred)
    print('k: ', k, '  score: ', scored)

#for k in k_range:
#    print('k: ', k, '  score: ', scores[k])

k:  2   score:  0.818093460657001
k:  3   score:  0.8326062811333333
k:  4   score:  0.830438999122699
k:  5   score:  0.8339410877820366
k:  6   score:  0.8317460565926712
k:  7   score:  0.8321971586264014
k:  8   score:  0.8302618312892625
k:  9   score:  0.8294820082151799


In [10]:
model = KNeighborsClassifier(n_neighbors = 5)

model.fit(X_train, y_train)

test = model.predict(X_test)

In [11]:
score = metrics.accuracy_score(y_test, test)
print(score)

0.8339410877820366


In [12]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state = 22)
model.fit(X_train, y_train)


DecisionTreeClassifier(random_state=22)

In [13]:
test = model.predict(X_test)

score = metrics.accuracy_score(y_test, test)

In [14]:
print(score)

0.8345316472268254


In [15]:
df_new = df.copy()
df_new = df_new.drop('ID', axis = 1)
df_new = df_new.drop('Source', axis =1)
df_new = df_new.drop(['Start_Time', 'End_Time', 'End_Lng', 'End_Lat', 'Number', 'Description', 'City', 'County'], axis = 1)
df_new.head()

,TMC,Severity,Start_Lat,Start_Lng,Distance(mi),Street,Side,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,201.0,3,39.865147,-84.058723,0.01,I-70 E,R,OH,45424,US,US/Eastern,KFFO,2016-02-08 05:58:00,36.9,NaN,91.0,29.68,10.0,Calm,NaN,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night
1,201.0,2,39.928059,-82.831184,0.01,Brice Rd,L,OH,43068-3402,US,US/Eastern,KCMH,2016-02-08 05:51:00,37.9,NaN,100.0,29.65,10.0,Calm,NaN,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Day
2,201.0,2,39.063148,-84.032608,0.01,State Route 32,R,OH,45176,US,US/Eastern,KI69,2016-02-08 06:56:00,36.0,33.3,100.0,29.67,10.0,SW,3.5,NaN,Overcast,False,False,False,False,False,False,False,False,False,False,False,True,False,Night,Night,Day,Day
3,201.0,3,39.747753,-84.205582,0.01,I-75 S,R,OH,45417,US,US/Eastern,KDAY,2016-02-08 07:38:00,35.1,31.0,96.0,29.64,9.0,SW,4.6,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Day,Day,Day
4,201.0,2,39.627781,-84.188354,0.01,Miamisburg Centerville Rd,R,OH,45459,US,US/Eastern,KMGY,2016-02-08 07:53:00,36.0,33.3,89.0,29.65,6.0,SW,3.5,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,True,False,Day,Day,Day,Day


In [16]:
#from sklearn.svm import SVR
#model = SVR(kernel = 'rbf')

#model.fit(X_train, y_train)

In [17]:
#test = model.predict(X_test)

#score = metrics.accuracy_score(y_test, test)

#print(score)

In [25]:
df['month'] = df['Start_Time'].dt.month
df['year'] = df['Start_Time'].dt.year
df['hour'] = df['Start_Time'].dt.hour
df['day'] = df['Start_Time'].dt.weekday

#print(df['year'][:10])

year_ticks = [2016, 2017, 2018, 2019, 2020]

In [28]:
df.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Description,Number,Street,Side,City,County,State,Zipcode,Country,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,month,year,hour,day
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,Right lane blocked due to accident on I-70 Eas...,NaN,I-70 E,R,Dayton,Montgomery,OH,45424,US,US/Eastern,KFFO,2016-02-08 05:58:00,36.9,NaN,91.0,29.68,10.0,Calm,NaN,0.02,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Night,2,2016,5,0
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,Accident on Brice Rd at Tussing Rd. Expect del...,2584.0,Brice Rd,L,Reynoldsburg,Franklin,OH,43068-3402,US,US/Eastern,KCMH,2016-02-08 05:51:00,37.9,NaN,100.0,29.65,10.0,Calm,NaN,0.00,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Night,Night,Day,2,2016,6,0
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,Accident on OH-32 State Route 32 Westbound at ...,NaN,State Route 32,R,Williamsburg,Clermont,OH,45176,US,US/Eastern,KI69,2016-02-08 06:56:00,36.0,33.3,100.0,29.67,10.0,SW,3.5,NaN,Overcast,False,False,False,False,False,False,False,False,False,False,False,True,False,Night,Night,Day,Day,2,2016,6,0
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,Accident on I-75 Southbound at Exits 52 52B US...,NaN,I-75 S,R,Dayton,Montgomery,OH,45417,US,US/Eastern,KDAY,2016-02-08 07:38:00,35.1,31.0,96.0,29.64,9.0,SW,4.6,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,Day,Day,Day,2,2016,7,0
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,Accident on McEwen Rd at OH-725 Miamisburg Cen...,NaN,Miamisburg Centerville Rd,R,Dayton,Montgomery,OH,45459,US,US/Eastern,KMGY,2016-02-08 07:53:00,36.0,33.3,89.0,29.65,6.0,SW,3.5,NaN,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,True,False,Day,Day,Day,Day,2,2016,7,0


In [27]:
print(df['day'].unique())

[0 1 2 3 4 5 6]


In [57]:
months = df.month
days = df.day
hours = df.hour

data = pd.DataFrame({'months': months, 'days': days, 'hours': hours, 'x': x, 'y': y})
#data = pd.DataFrame({'x': x, 'y': y})

#dat = [(x[i], y[i], months[i],days[i], hours[i]) for i in range(0, len(x))] 
#data_ = [data.coords, data.months.tolist(), data.days.tolist(), data.hours.tolist()]

In [55]:
X_train, X_test, y_train, y_test = train_test_split(data, sev, test_size=0.4, random_state=22)

In [56]:
model = KNeighborsClassifier(n_neighbors = 5)

model.fit(X_train, y_train)

test = model.predict(X_test)
score = metrics.accuracy_score(y_test, test)
print(score)

0.8339410877820366


In [64]:
df_test = df.copy()
df_test = df_test.drop(['Severity', 'ID'], axis = 1)

In [67]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth = 4)
X_train, X_test, y_train, y_test = train_test_split(data, sev, test_size=0.4, random_state=22)
model.fit(X_train, y_train)

RandomForestClassifier(max_depth=4)

In [68]:
test = model.predict(X_test)
score = metrics.accuracy_score(y_test, test)
print(score)

0.6812039159071811
